In [45]:
import pandas as pd
import numpy as np
import os

from bqplot import LogScale, LinearScale, OrdinalColorScale, ColorAxis, Axis, Scatter, Lines, CATEGORY10, Label, Figure, Tooltip

from ipywidgets import HBox, VBox, IntSlider, Play, jslink

In [46]:
initial_year = 1800

In [47]:
data = pd.read_json('nations.json')

In [48]:
data.head()

,name,region,income,population,lifeExpectancy
0,Angola,Sub-Saharan Africa,"[[1800, 359.93], [1820, 359.93], [1913, 556.12...","[[1800, 1567028], [1820, 1567028], [1940, 3738...","[[1800, 26.98], [1940, 26.98], [1950, 29.22], ..."
1,Benin,Sub-Saharan Africa,"[[1800, 553.72], [1820, 553.72], [1913, 855.53...","[[1800, 636559], [1820, 636559], [1950, 167266...","[[1800, 31], [1944, 31], [1950, 36.53], [1951,..."
2,Botswana,Sub-Saharan Africa,"[[1800, 407.36], [1820, 407.36], [1913, 629.4]...","[[1800, 121000], [1904, 121000], [1911, 125000...","[[1800, 33.6], [1945, 33.6], [1950, 46.82], [1..."
3,Burkina Faso,Sub-Saharan Africa,"[[1800, 454.33], [1820, 454.33], [1913, 497.44...","[[1800, 1665421], [1820, 1665421], [1950, 4376...","[[1800, 29.2], [1945, 29.2], [1950, 32.89], [1..."
4,Burundi,Sub-Saharan Africa,"[[1800, 447.59], [1820, 447.59], [1913, 353.82...","[[1800, 899097], [1820, 899097], [1950, 236252...","[[1800, 31.5], [1945, 31.5], [1950, 38.42], [1..."


In [49]:
def clean_data(data):
    for column in ['income', 'lifeExpectancy', 'population']:
        data = data.drop(data[data[column].apply(len) <= 4].index)
    return data

def extrap_interp(data):
    data = np.array(data)
    x_range = np.arange(1800, 2009, 1.)
    y_range = np.interp(x_range, data[:, 0], data[:, 1])
    return y_range

def extrap_data(data):
    for column in ['income', 'lifeExpectancy', 'population']:
        data[column] = data[column].apply(extrap_interp)
    return data

In [50]:
data = clean_data(data)
data = extrap_data(data)

In [51]:


income_min, income_max = np.min(data['income'].apply(np.min)), np.max(data['income'].apply(np.max))
life_exp_min, life_exp_max = np.min(data['lifeExpectancy'].apply(np.min)), np.max(data['lifeExpectancy'].apply(np.max))
pop_min, pop_max = np.min(data['population'].apply(np.min)), np.max(data['population'].apply(np.max))



In [52]:
def get_data(year):
    year_index = year - 1800
    income = data['income'].apply(lambda x: x[year_index])
    life_exp = data['lifeExpectancy'].apply(lambda x: x[year_index])
    pop =  data['population'].apply(lambda x: x[year_index])
    return income, life_exp, pop

In [53]:
tt = Tooltip(fields=['name', 'x', 'y'], labels=['Country Name', 'Income per Capita', 'Life Expectancy'])

In [54]:
year_label = Label(x=[0.75], y=[0.10], default_size=46, font_weight='bolder', colors=['orange'],
                   text=[str(initial_year)], enable_move=True)

In [55]:
x_sc = LogScale(min=income_min, max=income_max)
y_sc = LinearScale(min=life_exp_min, max=life_exp_max)
c_sc = OrdinalColorScale(domain=data['region'].unique().tolist(), colors=CATEGORY10[:6])
size_sc = LinearScale(min=pop_min, max=pop_max)

In [56]:


ax_y = Axis(label='Life Expectancy', scale=y_sc, orientation='vertical', side='left', grid_lines='solid')
ax_x = Axis(label='Income per Capita', scale=x_sc, grid_lines='solid')



In [57]:


# Start with the first year's data
cap_income, life_exp, pop = get_data(initial_year)



In [58]:
wealth_scat = Scatter(x=cap_income, y=life_exp, color=data['region'], size=pop,
                      names=data['name'], display_names=False,
                      scales={'x': x_sc, 'y': y_sc, 'color': c_sc, 'size': size_sc},
                      default_size=4112, tooltip=tt, animate=True, stroke='Black',
                      unhovered_style={'opacity': 0.5})

In [59]:
nation_line = Lines(x=data['income'][0], y=data['lifeExpectancy'][0], colors=['Gray'],
                       scales={'x': x_sc, 'y': y_sc}, visible=False)

In [60]:
time_interval = 10

In [61]:
fig = Figure(marks=[wealth_scat, year_label, nation_line], axes=[ax_x, ax_y],
             title='Health and Wealth of Nations', animation_duration=time_interval)

In [62]:
year_slider = IntSlider(min=1800, max=2008, step=1, description='Year', value=initial_year)

In [63]:
def hover_changed(change):
    if change.new is not None:
        nation_line.x = data[data['name'] == wealth_scat.names[change.new]]['income'].values[0]
        nation_line.y = data[data['name'] == wealth_scat.names[change.new]]['lifeExpectancy'].values[0]
        nation_line.visible = True
    else:
        nation_line.visible = False
        
wealth_scat.observe(hover_changed, 'hovered_point')

In [64]:


def year_changed(change):
    wealth_scat.x, wealth_scat.y, wealth_scat.size = get_data(year_slider.value)
    year_label.text = [str(year_slider.value)]

year_slider.observe(year_changed, 'value')



In [65]:
play_button = Play(min=1800, max=2008, interval=time_interval)
jslink((play_button, 'value'), (year_slider, 'value'))

Link(source=(Play(value=1800, interval=10, max=2008, min=1800), 'value'), target=(IntSlider(value=1800, descri…

In [66]:
VBox([HBox([play_button, year_slider]), fig])